## DV2599 Assignment 1
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [ ]:
# Import packages
import pandas as pd
import matplotlib.pyplot as mpl # Version 3.8.4
import seaborn as sb
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Load dataset
df = pd.read_csv('winequality-white.csv', delimiter=";")

#### 1. Inspect the dataset

In [29]:
# Describe
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


#### 2. Split into train and test data

In [30]:
# Set quality as target
y = df['quality']
x = df.drop('quality', axis=1)

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#### 3. Scaling

In [31]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform training data
x_train_scaled = scaler.fit_transform(x_train)

# Transform testing data
x_test_scaled = scaler.transform(x_test)

# Turn back into pandas dataframes
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)

# Describe
x_train_scaled.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000
mean,0.294001,0.194261,0.202124,0.186422,0.108696,0.225519,0.362184,0.297784,0.418273,0.312944,0.405714
std,0.080233,0.098998,0.073515,0.161473,0.063497,0.115402,0.118600,0.126331,0.140459,0.132393,0.198884
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.240385,0.127451,0.162651,0.035484,0.080119,0.139373,0.276923,0.197930,0.327103,0.220930,0.241935
50%,0.288462,0.176471,0.192771,0.148387,0.100890,0.216028,0.349650,0.284174,0.411215,0.290698,0.387097
75%,0.336538,0.235294,0.234940,0.300000,0.121662,0.299652,0.441958,0.388422,0.504673,0.383721,0.548387
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### 4. Repeated k-Fold Cross Validation

In [ ]:
# Initialize Repeated k-Fold Cross Validator
rkf = RepeatedKFold(n_splits=3, n_repeats=10)

# Random forest classifier and Logistic regression
classifiers = {
    "Random Forest" : RandomForestClassifier(),
    "Decision Tree" : DecisionTreeClassifier()
}

# Perform cross-validation
results = {}
for name, clf in classifiers.items():
    cv_scores = cross_val_score(clf, x_train_scaled, y_train, cv=rkf, scoring="accuracy")

    results[name] = {
        "average" : round(cv_scores.mean(), 4),
        "standard deviation" : round(cv_scores.std(), 4)
    }

results

{'Random Forest': {'average': 0.6464, 'standard deviation': 0.0132},
 'Logistic Regression': {'average': 0.5233, 'standard deviation': 0.0087},
 'Decision Tree': {'average': 0.5553, 'standard deviation': 0.0142}}

#### 5. Final model

In [33]:
# Random forest was best